<a href="https://colab.research.google.com/github/AllyHyeseongKim/2019_cau_oss_hackathon/blob/master/%ED%95%B4%EC%BB%A4%ED%86%A4%EC%9D%B4%EB%9D%BC%EC%93%B0%EA%B3%A0%EB%8C%80%ED%9A%8C%EB%9D%BC%EC%9D%BD%EB%8A%94%ED%98%9C%EC%84%B1%EC%9D%B4%EC%9D%98%EC%BC%80%EB%9D%BC%EC%8A%A4%EB%A7%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
 *  단, 사용할 데이터셋에 따라 is_mnist만 수정
*   모든 구현은 [2. 데이터 전처리]와 [3. 모델 생성]에서만 진행
 *  데이터 전처리 후 트레이닝, 데이터 셋은 x_train_after, x_test_after 변수명을 유지해주세요.
 *  데이터셋이 달라져도 같은 모델 구조를 유지하여야함.
*   [4. 모델 저장]과 [5. 모델 로드 및 평가]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *  트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
 *  team_name을 제외한 다른 부분은 수정하지 말 것
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  모델 구조 파일 (model_structure_teamXX.json)
 *  모델 weight 파일 (model_weight_teamXX.h5)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
* 제출 기한: **오후 6시**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2019_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  두 개의 데이터셋에 대해 다른 모델 구조를 사용한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

is_mnist = True;

# 데이터셋 로드
# x_train, y_train: 트레이닝 데이터 및 레이블
# x_test, y_test: 테스트 데이터 및 레이블
if is_mnist:
  data_type = 'mnist'
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() # fashion MNIST 데이터셋인 경우,
else:
  data_type = 'cifar10'
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() # cifar10 데이터셋인 경우,


# 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 총 클래스 개수
num_classes = y_test.shape[1]

# 인풋 데이터 타입
input_shape = x_test.shape[1:]

# **2. 데이터 전처리**


In [0]:
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

if is_mnist:
  x_train_after = x_train_after.reshape(x_train_after.shape[0], 28, 28, 1)
  x_test_after = x_test_after.reshape(x_test_after.shape[0], 28, 28, 1)
  input_shape = x_test_after.shape[1:]

# **3. 모델 생성**



In [39]:
#혜성
#from keras.preprocessing.image import ImageDataGenerator

# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# Convolution layer: 3 x 3 x 32 필터로 특징 추출
# 활성화 함수 = relu
model.add(keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'))

# MaxPooling layer: 출력 크기를 입력 크기의 반으로
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Dropout(0.25))

# Convolution layer: 3 x 3 x 64 필터로 특징 추출
# 활성화 함수 = relu
model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))

# MaxPooling layer: 출력 크기를 입력 크기의 반으로
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Dropout(0.25))

# Flatten layer: 7 x 7 x 1 image를 3136개의 1D vector input으로 변환
model.add(keras.layers.Flatten())

# 1st hidden layer: fully-connected layer
# (# of inputs = 3136, # of outputs = 256, actication fuction = relu)
model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dropout(0.5))

# 2nd hidden layer: fully-connected layer 
# (# of inputs = 256, # of outputs = 10, actication fuction = softmax)
model.add(keras.layers.Dense(10, activation='softmax'))

# learning rate scheduler 정의
# 30 epoch마다 learning rate을 0.1배
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import optimizers

def lr_scheduler(epoch, lr):
    decay_rate = 0.1
    decay_step = 90
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

save_path = '/content/'
team_name = 'team01'

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표

# optimizer를 Adam으로 설정
adam = optimizers.Adam(lr=0.001, decay=1e-5)
model_path = save_path + 'checkpoint' + data_type + '_' + team_name + '.h5'

# callback함수로 Checkpoint 객체 설정
cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='accuracy', verbose=1, save_best_only=True)

# callback함수 리스트 정의
callbacks=[
    cb_checkpoint, 
    LearningRateScheduler(lr_scheduler, verbose=1)
]

# 모델 호츨 및 컴파일 
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 128, epochs = 100, shuffle=True, validation_data=[x_test_after, y_test], callbacks=callbacks)

Train on 60000 samples, validate on 10000 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 1/100
59648/60000 [============================>.] - ETA: 0s - loss: 0.5303 - acc: 0.8076

W0824 06:48:08.360749 140464170198912 callbacks.py:989] Can save best model only with accuracy available, skipping.


60000/60000 [==============================] - 6s 102us/sample - loss: 0.5293 - acc: 0.8079 - val_loss: 0.3241 - val_acc: 0.8789

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 2/100
59648/60000 [============================>.] - ETA: 0s - loss: 0.3194 - acc: 0.8836

W0824 06:48:13.652235 140464170198912 callbacks.py:989] Can save best model only with accuracy available, skipping.


60000/60000 [==============================] - 5s 88us/sample - loss: 0.3195 - acc: 0.8835 - val_loss: 0.2711 - val_acc: 0.8992

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 3/100
59520/60000 [============================>.] - ETA: 0s - loss: 0.2710 - acc: 0.9011

...

60000/60000 [==============================] - 5s 88us/sample - loss: 0.0264 - acc: 0.9907 - val_loss: 0.3250 - val_acc: 0.9420

Epoch 00099: LearningRateScheduler reducing learning rate to 0.00010000000474974513.
Epoch 99/100
59520/60000 [============================>.] - ETA: 0s - loss: 0.0270 - acc: 0.9904

W0824 06:56:47.222740 140464170198912 callbacks.py:989] Can save best model only with accuracy available, skipping.


60000/60000 [==============================] - 5s 88us/sample - loss: 0.0269 - acc: 0.9905 - val_loss: 0.3228 - val_acc: 0.9413

Epoch 00100: LearningRateScheduler reducing learning rate to 0.00010000000474974513.
Epoch 100/100
59520/60000 [============================>.] - ETA: 0s - loss: 0.0252 - acc: 0.9912

W0824 06:56:52.487709 140464170198912 callbacks.py:989] Can save best model only with accuracy available, skipping.


60000/60000 [==============================] - 5s 88us/sample - loss: 0.0252 - acc: 0.9912 - val_loss: 0.3300 - val_acc: 0.9416


# **4. 모델 저장**

In [0]:
save_path = '/content/'
team_name = 'team01'

# 모델의 weight 값만 저장합니다.
model.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')

# 모델의 구조만을 저장합니다.
model_json = model.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_' + data_type + '_' + team_name + '.h5')

# **5. 모델 로드 및 평가**

In [41]:
save_path = '/content/'
team_name = 'team01'

model = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')
model.evaluate(x_test_after, y_test)

10000/10000 [==============================] - 1s 110us/sample - loss: 0.3300 - acc: 0.9416


[0.3299661362133455, 0.9416]